In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName("testApp").getOrCreate()

In [8]:
spark

In [94]:
df_spark=spark.read.csv("insuranceFraud.csv",inferSchema=True,header=True)
df_spark

DataFrame[claim_number: int, age_of_driver: int, gender: string, marital_status: double]

In [65]:
df_spark.show()

+------------+-------------+------+--------------+
|claim_number|age_of_driver|gender|marital_status|
+------------+-------------+------+--------------+
|           0|           50|     F|           1.0|
|           1|           47|  NULL|           1.0|
|           2|           28|     M|           0.0|
|           3|         NULL|     M|           1.0|
|           4|           60|     F|           1.0|
|           5|           23|     F|          NULL|
|           6|           79|     M|           1.0|
|           7|           12|     F|           0.0|
|           8|         NULL|     M|           1.0|
+------------+-------------+------+--------------+



In [29]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [30]:
df_spark.printSchema()

root
 |-- claim_number: integer (nullable = true)
 |-- age_of_driver: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- marital_status: double (nullable = true)



In [31]:
df_spark.columns

['claim_number', 'age_of_driver', 'gender', 'marital_status']

In [37]:
df_spark_reduced=df_spark.select(['age_of_driver','gender'])
df_spark_reduced.show()

+-------------+------+
|age_of_driver|gender|
+-------------+------+
|           50|     F|
|           47|     M|
|           28|     M|
|           36|     M|
|           60|     F|
+-------------+------+



In [42]:
df_spark.dtypes

[('claim_number', 'int'),
 ('age_of_driver', 'int'),
 ('gender', 'string'),
 ('marital_status', 'double')]

In [46]:
df_spark.describe().select(['summary','age_of_driver','marital_status']).show()

+-------+------------------+------------------+
|summary|     age_of_driver|    marital_status|
+-------+------------------+------------------+
|  count|                 5|                 5|
|   mean|              44.2|               0.8|
| stddev|12.457929201917949|0.4472135954999579|
|    min|                28|               0.0|
|    max|                60|               1.0|
+-------+------------------+------------------+



In [53]:
df_spark=df_spark.withColumn("age_after_2_years",df_spark["age_of_driver"]+2)
df_spark.show()

+------------+-------------+------+--------------+-----------------+
|claim_number|age_of_driver|gender|marital_status|age_after_2_years|
+------------+-------------+------+--------------+-----------------+
|           0|           50|     F|           1.0|               52|
|           1|           47|     M|           1.0|               49|
|           2|           28|     M|           0.0|               30|
|           3|           36|     M|           1.0|               38|
|           4|           60|     F|           1.0|               62|
+------------+-------------+------+--------------+-----------------+



In [55]:
drop_columns=['age_after_2_years']
for column in drop_columns:   
    df_spark=df_spark.drop(column)
df_spark.show()

+------------+-------------+------+--------------+
|claim_number|age_of_driver|gender|marital_status|
+------------+-------------+------+--------------+
|           0|           50|     F|           1.0|
|           1|           47|     M|           1.0|
|           2|           28|     M|           0.0|
|           3|           36|     M|           1.0|
|           4|           60|     F|           1.0|
+------------+-------------+------+--------------+



In [66]:
rename_dict={
    "age_of_driver":"age"
}

for old_name,new_name in rename_dict.items():
    df_spark=df_spark.withColumnRenamed(old_name,new_name)

df_spark.show()

+------------+----+------+--------------+
|claim_number| age|gender|marital_status|
+------------+----+------+--------------+
|           0|  50|     F|           1.0|
|           1|  47|  NULL|           1.0|
|           2|  28|     M|           0.0|
|           3|NULL|     M|           1.0|
|           4|  60|     F|           1.0|
|           5|  23|     F|          NULL|
|           6|  79|     M|           1.0|
|           7|  12|     F|           0.0|
|           8|NULL|     M|           1.0|
+------------+----+------+--------------+



In [72]:
df_spark.na.drop(how='all',subset=['age','gender']).show()

+------------+----+------+--------------+
|claim_number| age|gender|marital_status|
+------------+----+------+--------------+
|           0|  50|     F|           1.0|
|           1|  47|  NULL|           1.0|
|           2|  28|     M|           0.0|
|           3|NULL|     M|           1.0|
|           4|  60|     F|           1.0|
|           5|  23|     F|          NULL|
|           6|  79|     M|           1.0|
|           7|  12|     F|           0.0|
|           8|NULL|     M|           1.0|
+------------+----+------+--------------+



In [77]:
df_spark.na.fill(770,subset=['age','gender']).show()

+------------+---+------+--------------+
|claim_number|age|gender|marital_status|
+------------+---+------+--------------+
|           0| 50|     F|           1.0|
|           1| 47|  NULL|           1.0|
|           2| 28|     M|           0.0|
|           3|770|     M|           1.0|
|           4| 60|     F|           1.0|
|           5| 23|     F|          NULL|
|           6| 79|     M|           1.0|
|           7| 12|     F|           0.0|
|           8|770|     M|           1.0|
+------------+---+------+--------------+



In [95]:
from pyspark.ml.feature import Imputer

inputCols=['age_of_driver','marital_status']
outputCols=["{}_imputed".format(c) for c in inputCols]


imputer=Imputer(
    inputCols=inputCols,
    outputCols=outputCols
).setStrategy('mean')

In [96]:
imputer=imputer.fit(df_spark)
df_spark=imputer.transform(df_spark)
df_spark.show()

rename_dict={
    'age_of_driver_imputed':'age_of_driver',
    'marital_status_imputed':'marital_status'
}

for old_name,new_name in rename_dict.items() :
    df_spark=df_spark.drop(new_name)
    df_spark=df_spark.withColumnRenamed(old_name,new_name)

    
df_spark.show() 

+------------+-------------+------+--------------+---------------------+----------------------+
|claim_number|age_of_driver|gender|marital_status|age_of_driver_imputed|marital_status_imputed|
+------------+-------------+------+--------------+---------------------+----------------------+
|           0|           50|     F|           1.0|                   50|                   1.0|
|           1|           47|  NULL|           1.0|                   47|                   1.0|
|           2|           28|     M|           0.0|                   28|                   0.0|
|           3|         NULL|     M|           1.0|                   42|                   1.0|
|           4|           60|     F|           1.0|                   60|                   1.0|
|           5|           23|     F|          NULL|                   23|                  0.75|
|           6|           79|     M|           1.0|                   79|                   1.0|
|           7|           12|     F|     

In [97]:
df_spark.filter('claim_number = 0').show()

+------------+------+-------------+--------------+
|claim_number|gender|age_of_driver|marital_status|
+------------+------+-------------+--------------+
|           0|     F|           50|           1.0|
+------------+------+-------------+--------------+



In [102]:
df_spark.filter(
    (df_spark['age_of_driver']>45) &
    (df_spark['gender'] == 'F')
).show()

+------------+------+-------------+--------------+
|claim_number|gender|age_of_driver|marital_status|
+------------+------+-------------+--------------+
|           0|     F|           50|           1.0|
|           4|     F|           60|           1.0|
+------------+------+-------------+--------------+



In [105]:
df_spark.filter(
    (df_spark['age_of_driver']>50) |
    ((df_spark['gender'] == 'F') & (df_spark['marital_status']==0))
).show()

+------------+------+-------------+--------------+
|claim_number|gender|age_of_driver|marital_status|
+------------+------+-------------+--------------+
|           4|     F|           60|           1.0|
|           6|     M|           79|           1.0|
|           7|     F|           12|           0.0|
+------------+------+-------------+--------------+

